In [10]:
!pip install fuzzywuzzy

In [48]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pickle
from sklearn.feature_selection import VarianceThreshold
from fuzzywuzzy import process
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, log_loss
import warnings
warnings.filterwarnings('ignore')

import os
#Connecting the driver
from google.colab import drive

drive.mount('/content/drive/')

root_dir = '/content/drive/MyDrive/DAP/python_scripts'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler # to fit in features to single scale
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [53]:
def accuracy_measure(y_test, predictions):
    sensitivity_overall = recall_score(y_test, predictions, average='weighted')
    print('Sensitivity Overall Data:',sensitivity_overall)

    precision_overall = precision_score(y_test, predictions, average='weighted')
    print('Precision Overall Data:',precision_overall)

    f1_overall = f1_score(y_test, predictions, average='weighted')
    print('F1 Score Overall Data:',f1_overall)

    # Cohen's Kappa statistic
    kappa = cohen_kappa_score(y_test, predictions)
    print("Cohen's Kappa:", kappa)

    # Matthews Correlation Coefficient (MCC)
    mcc = matthews_corrcoef(y_test, predictions)
    print("Matthews Correlation Coefficient (MCC):", mcc)

    # Compute log loss
    #logloss = log_loss(y_test, predictions)
    #print(f"Log Loss: {logloss}")

    # Visualize predicted probabilities distribution
    #plt.figure(figsize=(8, 6))
    #sns.histplot(y_pred, bins=20, kde=True, color='skyblue')
    #plt.xlabel('Predicted Probabilities')
    #plt.ylabel('Frequency')
    #plt.title('Distribution of Predicted Probabilities')
    #plt.show()

###Random Forest without dimensionality reduction

In [54]:
df_training = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/training_dataset_final.csv")
X =df_training.drop(columns=['disease'])
y =df_training['disease']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# Print the shapes of the resulting sets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set: (8668, 404) (8668,)
Validation set: (1858, 404) (1858,)
Test set: (1858, 404) (1858,)


In [55]:
print('Random Forest without dimensionality reduction')
rfClassifier = RandomForestClassifier()
time_now = time.time()
rfClassifier.fit(X_train, y_train)
print('time taken: ',time.time()-time_now)

Random Forest without dimensionality reduction
time taken:  7.813596248626709


###By using multiple Dimensionality reduction techniques

In [56]:
def low_var():
    df_low_variance = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/low_variance_features.csv")
    df_low_variance= df_low_variance.fillna(0)
    df_low_variance.isna().any()
    X = df_low_variance.drop('disease', axis=1)
    y = df_low_variance['disease']
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X, y, X_train, y_train, X_test, X_val, y_test, y_val

def rand_forest():
    df_training = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/training_dataset_final.csv")
    df_feature = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/feature_importance_df.csv")
    df_feature= df_feature.fillna(0)
    df_training= df_training.fillna(0)

    feature_list=(df_feature['Feature'].head(100)).tolist()
    df_training_important_Features = df_training[feature_list]
    df_training_important_Features['disease'] = df_training['disease']
    df_training_important_Features.info()
    X =df_training_important_Features.drop(columns=['disease'])
    y =df_training_important_Features['disease']
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X, y, X_train, y_train, X_test, X_val, y_test, y_val

# Random Forest

In [57]:
def random_forest(X,y,X_train,y_train,X_test,X_val,y_test, y_val,pickle_file_name):
    rfClassifier = RandomForestClassifier()
    scaler = StandardScaler()
    X_scaled_train = scaler.fit_transform(X_train)
    X_scaled_test = scaler.transform(X_test)
    rfClassifier.fit(X_scaled_train, y_train)
    # Make predictions on the test set
    predictions = rfClassifier.predict(X_scaled_test)
    f1 = f1_score(y_test, predictions, average='weighted')
    print("F1 Score:", f1)

    # ----------------------------------------------------------------------
    param_grid = {
        'n_estimators': [25, 50, 100, 150, 200],
	      'max_features': ['sqrt', 'log2', None],
	      'max_depth': [3, 6, 9, None],
	      'max_leaf_nodes': [3, 6, 9, None],
}

    # Performing Grid Search using hyperparamters
    print('"####### Random Forest with grid search')
    param_test1 = {'n_estimators':200,'max_features':'log2'}
    clf = GridSearchCV(rfClassifier, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)
    best_clf = clf.fit(X_train, y_train)

    cv_results = pd.DataFrame(best_clf.cv_results_)
    print (f'Accuracy: {best_clf.score(X,y):.3f}')
    print(best_clf)
    best_params = best_clf.best_params_
    print("Best hyperparameters:", best_clf.best_params_)
    print("Best model:", best_clf.best_estimator_)

    best_params = best_clf.best_params_
    # Create a Random Forest model with the best hyperparameters
    #best_rf_model = rfClassifier(**best_params)
    best_rf_model= best_clf.best_estimator_
# Train the model on the training data
    model = best_rf_model.fit(X_train, y_train)

    # Make predictions on the testing data
    predictions = best_rf_model.predict(X_test)

    # Evaluate the performance of the model
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy score:", accuracy)
    print('####### Cross Validation')
    cv_scores = cross_val_score(rfClassifier, X_test, y_test, cv=5)
    print('Scores from cross-validation is: ', cv_scores)
    print('Average accuracy from cross-validation is: {}'.format(np.mean(cv_scores)))

    accuracy_measure(y_test, predictions)
    print("----------------------------------------------------------------------")

    ### Validating the model
    print( "####### Validating the model" )
    predicted_class_validation = best_rf_model.predict(X_val)
    print("Predicted class:", predicted_class_validation)

    accuracy = accuracy_score(y_val, predicted_class_validation)
    print("Accuracy on the validation set:", accuracy)
    accuracy_measure(y_val, predicted_class_validation)

    ### Saving the model with best parameters random forest
    print( "####### Saving the model with best parameters random forest" )
    # Save the model to a file using pickle
    with open(pickle_file_name, 'wb') as file:
        pickle.dump(model, file)



In [58]:
X,y,X_train,y_train,X_test,X_val,y_test, y_val = low_var()

In [61]:
file_name = os.path.join(root_dir,'low_var_features_rf.txt')
with open(file_name, 'w') as fp:
    for item in X_train:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [62]:
random_forest(X,y,X_train,y_train,X_test,X_val,y_test, y_val,'random_forest_model_lv.pkl')

F1 Score: 0.9730718366073234
"####### Random Forest with grid search
Fitting 3 folds for each of 240 candidates, totalling 720 fits
Accuracy: 0.991
GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, None],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [3, 6, 9, None],
                         'n_estimators': [25, 50, 100, 150, 200]},
             verbose=True)
Best hyperparameters: {'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'n_estimators': 25}
Best model: RandomForestClassifier(max_features=None, n_estimators=25)
Accuracy score: 0.9677072120559742
####### Cross Validation
Scores from cross-validation is:  [0.92741935 0.90322581 0.91935484 0.88948787 0.89757412]
Average accuracy from cross-validation is: 0.9074123989218329
Sensitivity Overall Data: 0.9677072120559742
Precision Overall Data: 0.9709950428855058
F1 Score Overall Data: 0.9676151

Random Forest after dimensionality reduction with Random Forset

In [63]:
X,y,X_train,y_train,X_test,X_val,y_test, y_val = rand_forest()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12384 entries, 0 to 12383
Columns: 101 entries, pain to disease
dtypes: float64(100), object(1)
memory usage: 9.5+ MB


In [64]:
file_name = os.path.join(root_dir,'rand_for_features_rf.txt')
with open(file_name, 'w') as fp:
    for item in X_train:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [65]:
random_forest(X,y,X_train,y_train,X_test,X_val,y_test, y_val,'random_forest_model_rd.pkl')

F1 Score: 0.8336679710609051
"####### Random Forest with grid search
Fitting 3 folds for each of 240 candidates, totalling 720 fits
Accuracy: 0.855
GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, None],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [3, 6, 9, None],
                         'n_estimators': [25, 50, 100, 150, 200]},
             verbose=True)
Best hyperparameters: {'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None, 'n_estimators': 200}
Best model: RandomForestClassifier(max_features='log2', n_estimators=200)
Accuracy score: 0.8304628632938643
####### Cross Validation
Scores from cross-validation is:  [0.80913978 0.76344086 0.80107527 0.77897574 0.81132075]
Average accuracy from cross-validation is: 0.7927904819870736
Sensitivity Overall Data: 0.8304628632938643
Precision Overall Data: 0.8697198309701445
F1 Score Overall Data: 0.8

###Predicting the Disease

In [67]:
# getting primitive care
df_primitive_care = pd.read_csv(os.path.join(root_dir,"prec_t.csv"))
diagnose_list = df_primitive_care['diagnose'].tolist()
df_primitive_care.head(3)

,did,diagnose,pid
0,1,Abdominal aortic aneurysm: (enlarged major blo...,Stopping smoking may slow their growth. Small ...
1,2,Abdominal swelling,Eating smaller meals may help prevent bloating...
2,3,Abdominal trauma,Surgery.


In [68]:
def predicting_disease(syp,diagnose_list,feature_names):
    columns_list=feature_names.copy()
    for col in syp:
        if col not in feature_names:
            #print(col)
            syp.remove(col)

    temp_df = pd.DataFrame(0, index=[0], columns=columns_list)
    temp_df.loc[0, syp] = 1
    print(syp)
    print(temp_df)
    predicted_class_1 = loaded_model.predict(temp_df)
    #print("Predicted class:", predicted_class_1)
    # getting primitive care
    predicted_class_1 = str(predicted_class_1)
    diagnose_list = [str(item) for item in diagnose_list]
    best_match, similarity_score = process.extractOne(predicted_class_1, diagnose_list)
    threshold = 70
    if similarity_score >= threshold:
        print(f"Best match for '{predicted_class_1}': '{best_match}' with similarity score: {similarity_score}")
        primitive_care= df_primitive_care[df_primitive_care['diagnose']==best_match]
        primitive_care = primitive_care['pid'].iloc[0]
        message = f"Based on the symptoms you entered, there are chances that you are suffering from **{predicted_class_1}**.\n"\
          f"\nThe primitive care we would suggest is **{primitive_care}** \nThis recommendation is based on the symptoms you provided.\n "\
        "However, please note that this is not a substitute for professional medical advice. We strongly recommend consulting a healthcare professional "\
          "for further clarification and a proper diagnosis. Your health is important, and a medical professional can provide personalized guidance and care."
        print(message)

    else:
        print(f" Diseases Predicted '{predicted_class_1}' in the is not monitored.")

In [69]:
feature_names = X_train.columns.tolist()
syp = ['facial paresis','muscle twitch','chill','fever','cough','decreased body weight']

In [70]:
#predicting_disease(syp,diagnose_list,feature_names)

['facial paresis', 'chill', 'fever', 'cough', 'decreased body weight']
   pain  fever  lesion  shortness of breath  mass of body structure  nausea  \
0     0      1       0                    0                       0       0   

   pain abdominal  pain chest  night sweat  diarrhea  ...  general discomfort  \
0               0           0            0         0  ...                   0   

   energy increased  dysesthesia  feeling hopeless  focal seizures  hirsutism  \
0                 0            0                 0               0          0   

   hoarseness  hypoxemia  green sputum  polymyalgia  
0           0          0             0            0  

[1 rows x 100 columns]


NameError: ignored

In [ ]:
syp = ['abscess bacterial','muscle twitch','chill','fever','barking cough']
predicting_disease(syp,diagnose_list,feature_names)
# in this case "abscess bacterial" is not the correct syp